In [1]:
import time
import datetime
from functools import partial

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from scipy import ndimage

from model import conv_block
from data import example_to_tensor, normalize, add_channel_axis
from plot import plot_slice, plot_animated_volume
from config import CT_0, CT_1, CT_2, CT_3, SEED, SCAN_SHAPE

%matplotlib inline
plt.rcParams["figure.figsize"] = [15, 7]

In [27]:
tf.random.set_seed(SEED)

input_shape = (*SCAN_SHAPE, 1)

epochs = 1000
patience = 50
batch_size = 8
learning_rate = 0.00001
dropout_rate = 0.5
val_perc = 0.12  # percentage from the already splitted training test
test_perc = 0.1

In [28]:
@tf.function
def one_hot_four_classes(x):
    return tf.one_hot(x, depth=4, dtype=tf.float32)

In [29]:
def load_dataset(tfrecord_fname: str, label, cardinality=None):
    """Return a tensorflow Dataset (scan, one_hot_vector).

    If cardinality is provided, the function will run faster.
    """
    x_dataset = (
        tf.data.TFRecordDataset(tfrecord_fname)
        .map(example_to_tensor, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        .map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        .map(add_channel_axis, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    )
    if not cardinality:
        cardinality = sum(1 for _ in x_dataset)
    y_dataset = (
        tf.data.Dataset.from_tensor_slices([label])
        .repeat(cardinality)
        .map(one_hot_four_classes, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    )
    return tf.data.Dataset.zip((x_dataset, y_dataset))

In [30]:
@tf.function
def random_rotate(volume, label):
    "Rotate the volume by a random degree"

    def scipy_rotate(volume):
        angle = tf.random.uniform(shape=(1,), minval=-180, maxval=180, dtype=tf.int32)[
            0
        ].numpy()
        volume = ndimage.rotate(volume, angle, axes=(1, 2), reshape=False)
        volume[volume < 0] = 0
        volume[volume > 1] = 1
        return volume

    augmented_volume = tf.numpy_function(scipy_rotate, [volume], tf.float32)
    return augmented_volume, label

In [31]:
@tf.function
def random_flip(volume, label):
    if tf.random.uniform((1,), minval=0, maxval=2, dtype=tf.int32)[0] == 1:
        volume = tf.reverse(volume, axis=(1,))
    return volume, label

In [18]:
def set_shape(volume, label):
    volume.set_shape(input_shape)
    return volume, label

In [19]:
ct_0_train_dataset = load_dataset(
    CT_0.TRAIN_TFRECORD,
    label=0,
    cardinality=CT_0.RESAMPLED_TRAIN_SIZE,
)
ct_1_train_dataset = load_dataset(
    CT_1.TRAIN_TFRECORD,
    label=1,
    cardinality=CT_1.RESAMPLED_TRAIN_SIZE,
)
ct_2_train_dataset = load_dataset(
    CT_2.TRAIN_TFRECORD,
    label=2,
    cardinality=CT_2.RESAMPLED_TRAIN_SIZE,
)
ct_3_train_dataset = load_dataset(
    CT_3.TRAIN_TFRECORD,
    label=3,
    cardinality=CT_3.RESAMPLED_TRAIN_SIZE,
)
train_dataset = (
    ct_0_train_dataset.concatenate(ct_1_train_dataset)
    .concatenate(ct_2_train_dataset)
    .concatenate(ct_3_train_dataset)
    .shuffle(
        buffer_size=(
            CT_0.RESAMPLED_TRAIN_SIZE
            + CT_1.RESAMPLED_TRAIN_SIZE
            + CT_2.RESAMPLED_TRAIN_SIZE
            + CT_3.RESAMPLED_TRAIN_SIZE
        ),
        seed=SEED,
        reshuffle_each_iteration=False,
    )
    .map(random_rotate, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(random_flip, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    # Set the volume shape because after a tf.numpy_function it loses the shape
    .map(lambda x, y: set_shape(x, y), num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .batch(batch_size)
    .cache()  # must be called before shuffle
    .shuffle(buffer_size=64, reshuffle_each_iteration=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)
train_dataset

Cause: could not parse the source code:

    .map(lambda x, y: set_shape(x, y), num_parallel_calls=tf.data.experimental.AUTOTUNE)

This error may be avoided by creating the lambda in a standalone statement.

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code:

    .map(lambda x, y: set_shape(x, y), num_parallel_calls=tf.data.experimental.AUTOTUNE)

This error may be avoided by creating the lambda in a standalone statement.

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


<PrefetchDataset shapes: ((None, 48, 256, 256, 1), (None, 4)), types: (tf.float32, tf.float32)>

In [20]:
ct_0_val_dataset = load_dataset(
    CT_0.VAL_TFRECORD,
    label=0,
    cardinality=CT_0.VAL_SIZE,
)
ct_1_val_dataset = load_dataset(
    CT_1.VAL_TFRECORD,
    label=1,
    cardinality=CT_1.VAL_SIZE,
)
ct_2_val_dataset = load_dataset(
    CT_2.VAL_TFRECORD,
    label=2,
    cardinality=CT_2.VAL_SIZE,
)
ct_3_val_dataset = load_dataset(
    CT_3.VAL_TFRECORD,
    label=3,
    cardinality=CT_3.VAL_SIZE,
)
val_dataset = (
    ct_0_val_dataset.concatenate(ct_1_val_dataset)
    .concatenate(ct_2_val_dataset)
    .concatenate(ct_3_val_dataset)
    .shuffle(
        buffer_size=(CT_0.VAL_SIZE + CT_1.VAL_SIZE + CT_2.VAL_SIZE + CT_3.VAL_SIZE),
        seed=SEED,
        reshuffle_each_iteration=False,
    )
    .batch(batch_size)
    .cache()
    .prefetch(tf.data.experimental.AUTOTUNE)
)
val_dataset

<PrefetchDataset shapes: ((None, None, None, None, 1), (None, 4)), types: (tf.float32, tf.float32)>

In [21]:
ct_0_test_dataset = load_dataset(
    CT_0.TEST_TFRECORD,
    label=0,
    cardinality=CT_0.TEST_SIZE,
)
ct_1_test_dataset = load_dataset(
    CT_1.TEST_TFRECORD,
    label=1,
    cardinality=CT_1.TEST_SIZE,
)
ct_2_test_dataset = load_dataset(
    CT_2.TEST_TFRECORD,
    label=2,
    cardinality=CT_2.TEST_SIZE,
)
ct_3_test_dataset = load_dataset(
    CT_3.TEST_TFRECORD,
    label=3,
    cardinality=CT_3.TEST_SIZE,
)
test_dataset = (
    ct_0_test_dataset.concatenate(ct_1_test_dataset)
    .concatenate(ct_2_test_dataset)
    .concatenate(ct_3_test_dataset)
    .shuffle(
        buffer_size=(CT_0.TEST_SIZE + CT_1.TEST_SIZE + CT_2.TEST_SIZE + CT_3.TEST_SIZE),
        seed=SEED,
        reshuffle_each_iteration=False,
    )
    .batch(batch_size)
)
test_dataset

<BatchDataset shapes: ((None, None, None, None, 1), (None, 4)), types: (tf.float32, tf.float32)>

In [22]:
SeluConv3D = partial(
    keras.layers.Conv3D,
    padding="same",
    kernel_initializer="lecun_normal",
    bias_initializer="zeros",
    activation="selu",
)

In [23]:
SeluDense = partial(
    keras.layers.Dense,
    kernel_initializer="lecun_normal",
    bias_initializer="zeros",
    activation="selu",
)

In [39]:
cnn = keras.Sequential(
    [
        keras.layers.InputLayer(input_shape),
        SeluConv3D(filters=32, kernel_size=3),
        keras.layers.AlphaDropout(0.0),
        keras.layers.MaxPooling3D(pool_size=2),
        SeluConv3D(filters=64, kernel_size=3),
        keras.layers.AlphaDropout(0.0),
        keras.layers.MaxPooling3D(pool_size=2),
        SeluConv3D(filters=128, kernel_size=3),
        keras.layers.AlphaDropout(0.0),
        keras.layers.MaxPooling3D(pool_size=2),
        SeluConv3D(filters=128, kernel_size=3),
        keras.layers.AlphaDropout(0.0),
        keras.layers.MaxPooling3D(pool_size=2),
        keras.layers.Flatten(),
        SeluDense(units=512),
        keras.layers.AlphaDropout(0.5),
        keras.layers.Dense(4, activation="softmax"),
    ],
    name="3dcnn",
)
cnn.summary()

Model: "3dcnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_16 (Conv3D)           (None, 48, 256, 256, 32)  896       
_________________________________________________________________
alpha_dropout_20 (AlphaDropo (None, 48, 256, 256, 32)  0         
_________________________________________________________________
max_pooling3d_16 (MaxPooling (None, 24, 128, 128, 32)  0         
_________________________________________________________________
conv3d_17 (Conv3D)           (None, 24, 128, 128, 64)  55360     
_________________________________________________________________
alpha_dropout_21 (AlphaDropo (None, 24, 128, 128, 64)  0         
_________________________________________________________________
max_pooling3d_17 (MaxPooling (None, 12, 64, 64, 64)    0         
_________________________________________________________________
conv3d_18 (Conv3D)           (None, 12, 64, 64, 128)   221312

In [40]:
cnn.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [ ]:
monitor_metric = "val_accuracy"

start_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
best_checkpoint = f"models/baseline-{start_time}.h5"
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    best_checkpoint,
    monitor=monitor_metric,
    mode="max",
    verbose=1,
    save_best_only=True,
)
early_stopping_cb = keras.callbacks.EarlyStopping(
    monitor=monitor_metric, patience=patience, mode="max"
)
log_dir = f"logs/baseline-{start_time}"
file_writer = tf.summary.create_file_writer(log_dir)
with file_writer.as_default():
    tf.summary.text(
        "Hyperparameters",
        f"{SEED=}; "
        f"{input_shape=}; "
        f"{epochs=}; "
        f"{patience=}; "
        f"{batch_size=}; "
        f"{learning_rate=}; "
        f"{dropout_rate=}; "
        f"{val_perc=}; "
        f"{test_perc=}",
        step=0,
    )
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=False,
    profile_batch=0,
)
cnn.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb],
)

Epoch 1/1000
      2/Unknown - 0s 216ms/step - loss: 2.4258 - accuracy: 0.4375WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1562s vs `on_train_batch_end` time: 0.2756s). Check your callbacks.
    113/Unknown - 49s 430ms/step - loss: 1.7669 - accuracy: 0.3068
Epoch 00001: val_accuracy improved from -inf to 0.23164, saving model to models/baseline-20201106-203356.h5
113/113 [==============================] - 110s 973ms/step - loss: 1.7669 - accuracy: 0.3068 - val_loss: 1.6687 - val_accuracy: 0.2316
Epoch 2/1000
113/113 [==============================] - ETA: 0s - loss: 1.3710 - accuracy: 0.3843
Epoch 00002: val_accuracy improved from 0.23164 to 0.42373, saving model to models/baseline-20201106-203356.h5
113/113 [==============================] - 110s 972ms/step - loss: 1.3710 - accuracy: 0.3843 - val_loss: 1.2178 - val_accuracy: 0.4237
Epoch 3/1000
113/113 [==============================] - ETA: 0s - loss: 1.2595 - accuracy: 0.

In [ ]:
cnn = keras.models.load_model("models/baseline-20201029-111058.h5")
cnn.evaluate(test_dataset, verbose=1, return_dict=True)

In [ ]:
cnn = keras.models.load_model("models/baseline-20201029-113438.h5")
cnn.evaluate(test_dataset, verbose=1, return_dict=True)

In [ ]:
cnn = keras.models.load_model("models/baseline-20201029-115235.h5")
cnn.evaluate(test_dataset, verbose=1, return_dict=True)

In [ ]:
x, y = next(iter(test_dataset.skip(5)))
prediction = cnn(x, training=False)
print(f"real: {y.numpy()}, prediction: {prediction.numpy()}")
plot_animated_volume(x[0, :], fps=3)

In [ ]:
def prediction_bias(dataset):
    """Prediction bias is the difference
        average_labels - average_predictions

    It should be near zero.
    Return the tuple (label_avg, prediction_avg, prediction_bias)
    """
    label_avg = np.mean([label.numpy()[0] for _, label in dataset.unbatch()])

    def gen():
        for x, _ in dataset:
            yield x

    x_dataset = (
        tf.data.Dataset.from_generator(gen, tf.float32)
        .unbatch()
        .padded_batch(1, input_shape)
    )
    prediction_avg = np.mean([cnn(x, training=False).numpy()[0][0] for x in x_dataset])
    return label_avg, prediction_avg, np.abs(label_avg - prediction_avg)

In [ ]:
l, p, b = prediction_bias(train_dataset)
print(f"Labels average: {l}")
print(f"Predictions average: {p}")
print(f"Prediction bias: {b}")